<a href="https://colab.research.google.com/github/daniel-afram/project-alura-google-AI/blob/main/Project_Alura_Google_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai
import google.generativeai as genai
from google.colab import userdata
import random
import time

api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

# Encontrar um modelo que suporte geração de conteúdo
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    m.name

# Configurações de geração e segurança
generation_config = {
    "candidate_count": 1,
    "temperature": 0.8,
}

safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

# Inicializando o modelo Gemini
model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    generation_config=generation_config,
    safety_settings=safety_settings
)

# Função para gerar saudações variadas
def saudacao():
  saudacoes = [
      "Olá! 👋 É um prazer te ajudar a encontrar algo legal para assistir, ler ou jogar!",
      "E aí! 😄 Precisa de recomendações de filmes, séries, livros ou jogos? Pode contar comigo!",
      "Oi, oi! 😊 Qual mídia você gostaria de explorar hoje?",
  ]
  return random.choice(saudacoes)

# Prompt inicial de boas-vindas
prompt_inicial = f"{saudacao()}"

print(prompt_inicial)

# Histórico da conversa
histórico_conversa = [
    {"role": "assistant", "content": prompt_inicial}
]

# Prompt inicial de boas-vindas
prompt_base_conhecimento = """
Gostaria de uma recomendação ou análise de um filme, série, anime, livro ou jogo. Por favor, me diga:
Tipo de mídia: (filme, série, anime, livro, jogo)
Ação: (recomendação ou análise)
Detalhes:
Para recomendações: Especifique o gênero, tema ou características desejadas.
Para análises: Indique o título da obra.
Com base em sua solicitação, o relatório incluirá:
Informações principais: Título, gênero, ano de lançamento, autor/diretor, plataformas disponíveis, duração média (para jogos), classificação indicativa.
Avaliação:
Notas de sites especializados: IMDb, Rotten Tomatoes, Metacritic para filmes, séries, livros e animes. Metacritic, Steam, Playstation Store, Xbox Store, Nintendo eShop para jogos (especificar a plataforma para cada jogo).
Notas de usuários (média das avaliações dos usuários).
Análise de 10 comentários de usuários, com diferentes perspectivas sobre a obra.
Sinopse: Descrição concisa da trama.
Análise: Pontos fortes, pontos fracos, temas abordados, impacto cultural, relevância histórica.
Elenco principal: Atores, dubladores (para jogos e animes).
Ordem cronológica:
Mídia principal:
Se for parte de uma saga, apresentar a ordem cronológica de todas as obras da saga, incluindo a mídia solicitada.
Detalhar cada obra da saga, com título, número de episódios, ano de lançamento, classificação indicativa e breve sinopse.
Mídias relacionadas: Mostrar a ordem cronológica de lançamento das outras mídias que expandem o universo da obra (ex: filmes, séries, livros) com título, ano de lançamento, classificação indicativa e breve sinopse.
Mídias relacionadas: Filmes, séries, animes ou jogos que expandem o universo da obra.
O Relatório deve ter emojis relacionados com todos os títulos dos tópicos mencionados e alguns emojis para os comentários dos usuários.
Recomendações: "Se você gosta desse...você também vai gostar desses:" 5 sugestões personalizadas de acordo com suas preferências, considerando seus pedidos anteriores.
Exemplos:
Exemplo 1:
Tipo de mídia: Filme
Ação: Análise
Detalhes: Star Wars Episódio VII: O Despertar da Força
Relatório: O relatório deve apresentar a ordem cronológica de TODOS os filmes da saga Star Wars, incluindo Episódio VII, e detalhar cada filme com título, ano, classificação indicativa e sinopse. Também deve apresentar a ordem cronológica e detalhes de outros filmes relacionados, como "Rogue One" e "Solo", incluindo suas classificações indicativas.
Exemplo 2:
Tipo de mídia: Jogo
Ação: Análise
Detalhes: The Witcher 3: Wild Hunt
Relatório: O relatório deve apresentar a ordem cronológica dos jogos da série The Witcher (incluindo spin-offs), com detalhes de cada jogo, incluindo a classificação indicativa. A ordem cronológica deve considerar a história do universo, não apenas a data de lançamento dos jogos. Também deve apresentar a ordem cronológica dos livros da saga The Witcher, que originou o universo do jogo, com suas respectivas classificações indicativas.
"""

# Histórico da conversa
histórico_conversa = [
    {"role": "assistant", "content": prompt_base_conhecimento}
]

# Loop principal do chatbot
while True:
  prompt = input("O que deseja? (ou 'fim' para sair): ")
  if prompt.lower() == 'fim':
    break

  # Adicionando a mensagem do usuário ao histórico
  histórico_conversa.append({"role": "user", "content": prompt})

  # Construindo o prompt completo com o histórico
  prompt_completo = "".join(
      [f"{mensagem['role']}: {mensagem['content']}\n" for mensagem in histórico_conversa]
  )

  # Gerando a resposta com o modelo
  response = model.generate_content(
      prompt_completo + "assistant:",
      generation_config=generation_config,
      safety_settings=safety_settings
  )

  # Adicionando a resposta do assistente ao histórico
  histórico_conversa.append({"role": "assistant", "content": response.text})

  print("Resposta:\n", response.text, "\n")